#3주차 과제
1. MNIST 데이터셋으로 분류기를 만들어 테스트 세트에서 97% 정확도를 달성해보세요.
힌트:  KNeigborsClassifier를 활용해보고, 좋은 하이퍼파라미터 값만 찾으면 됩니다. (weights와 n_neighbors 하이퍼파라미터로 그리드 탐색을 시도해보세요.)

**수행 단계 요약**
1. **데이터 증강**: 훈련 데이터를 상하좌우로 1픽셀씩 shift해서 훈련 셋을 30만개까지 늘림
2. **KNeighborsClassifier weights 조정**: 그리드 탐색으로 이웃 가중치를 조정(uniform, weights)
3. **KNeighborsClassifier n_neighbors 조정**: 그리드 탐색으로 이웃 수를 조정(k=1~9)
4. **PCA variance_ratios 조정**: 그리드 탐색으로 PCA의 분산 보존 비율을 조정(0.7~0.9)

파이프라인 함수 정의

In [9]:
# 1. 라이브러리 임포트
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
import operator
from scipy.ndimage import shift

이미지 이동(shift) 함수: 데이터 증강에 사용

In [10]:
# 2.1. 헬퍼 함수: shift_image
def shift_image(image, dx, dy):
    """
    이미지를 (dx, dy)만큼 이동시키는 헬퍼 함수
    """
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode='constant') 
    return shifted_image.reshape([-1])

데이터 로드 및 분할

In [11]:
# 2.2. 1단계: 데이터 로드 및 분할 함수
def load_and_split_data():
    """
    MNIST 데이터셋을 로드하고 훈련/테스트 세트로 분할합니다.
    """
    print("MNIST 전체 데이터셋 로드 중...")
    mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')
    X, y = mnist["data"], mnist["target"].astype(int)
    
    X_train, X_test = X[:60000], X[60000:]
    y_train, y_test = y[:60000], y[60000:]
    
    print(f"원본 훈련 세트: {len(y_train)}개, 테스트 세트: {len(y_test)}개")
    return X_train, y_train, X_test, y_test

데이터 증강: 훈련 세트(6만개)를 상하좌우로 1픽셀씩 이동시켜서 30만개의 데이터로 증강시킴

In [12]:
# 2.3. 2단계: 데이터 증강 함수
def augment_data(X_train, y_train):
    """
    훈련 세트를 상/하/좌/우로 1픽셀씩 이동시켜 5배로 증강합니다.
    """
    print("\n데이터 증강 시작...")
    X_train_augmented = [X_train]
    y_train_augmented = [y_train]
    shifts = [(0, 1), (0, -1), (1, 0), (-1, 0)] 
    
    for dy, dx in shifts:
        print(f"  ... (dy={dy}, dx={dx}) 이동 적용 중 ...")
        X_shifted = np.apply_along_axis(shift_image, axis=1, arr=X_train, dx=dx, dy=dy)
        X_train_augmented.append(X_shifted)
        y_train_augmented.append(y_train)
        
    X_train_augmented_np = np.vstack(X_train_augmented)
    y_train_augmented_np = np.concatenate(y_train_augmented)
    
    print(f"총 훈련 세트 크기: {len(y_train_augmented_np)}개")
    return X_train_augmented_np, y_train_augmented_np

스케일링

In [13]:
# 2.4. 3단계: 스케일링 함수
def scale_data(X_train_aug, X_test):
    """
    StandardScaler를 사용하여 증강된 훈련 세트로 fit하고
    훈련/테스트 세트를 transform합니다.
    """
    print("\n데이터 스케일링 중...")
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_aug)
    X_test_scaled = scaler.transform(X_test)
    print("스케일링 완료.")
    return X_train_scaled, X_test_scaled

PCA, KNN 튜닝: 그리드 탐색으로 'weights', 'n_neighbors', 'variance'를 튜닝
 - weights: 'uniform' or 'weight'
 - n_neighbors: k=2,4,6,8
 - vairance: 0.65, 0.75, 0.85, 0.95

In [14]:
# 2.5. 4단계: PCA 및 KNN 튜닝 함수
def tune_pca_and_knn(X_train_scaled, y_train_aug, X_test_scaled, y_test, 
                     variance_ratios, weights_options, k_values): 
    """
    주어진 모든 하이퍼파라미터 조합(3중 루프)으로 그리드 탐색을 수행합니다.
    """
    results = [] 
    print(f"\n--- PCA 및 KNN 그리드 탐색 시작 ---")
    print(f"후보 분산: {variance_ratios}")
    print(f"후보 Weights: {weights_options}")
    print(f"후보 K: {k_values}")
    total_runs = len(weights_options) * len(variance_ratios) * len(k_values)
    print(f"(총 {total_runs}회 실행. 매우 오래 걸립니다.)")

    best_accuracy_so_far = 0.0
    run_count = 0

    # 1. Weights 루프 (바깥쪽)
    for w_mode in weights_options: 
        
        # 2. 분산 비율 루프 (중간)
        for var_ratio in variance_ratios: 
            
            print(f"\n[ Weights = '{w_mode}', 분산 = {var_ratio} ] 테스트 시작...")
            
            # 2-1. PCA 적용
            print(f"  PCA 적용 (n_components={var_ratio})...")
            
            pca = PCA(n_components=var_ratio, random_state=42) 
            X_train_reduced = pca.fit_transform(X_train_scaled)
            X_test_reduced = pca.transform(X_test_scaled) 
            n_components_found = pca.n_components_
            
            print(f"  PCA 완료. (차원 수: {n_components_found}개)")

            # 3. K 루프 (안쪽)
            for k in k_values:
                run_count += 1
                print(f"  [ {run_count}/{total_runs} ] k={k} 모델 훈련/예측 중...")
                
                knn_clf = KNeighborsClassifier(n_neighbors=k,
                                               weights=w_mode, 
                                               n_jobs=-1)
                
                knn_clf.fit(X_train_reduced, y_train_aug)
                y_pred = knn_clf.predict(X_test_reduced) 
                
                print(f"  KNN 완료.")
                
                # 3-3. 결과 저장 및 '최고 모델' 갱신
                accuracy = accuracy_score(y_test, y_pred)
                print(f"  >>> (k={k}) 정확도: {accuracy * 100:.2f}% <<<")
                
                current_result = {
                    'k': k,
                    'weights': w_mode, 
                    'variance': var_ratio, 
                    'n_components': n_components_found, 
                    'accuracy': accuracy
                }
                results.append(current_result)
                
                if accuracy > best_accuracy_so_far:
                    print(f"  *** 새 최고 정확도 달성! ***")
                    best_accuracy_so_far = accuracy
                
    return results # 결과 리스트만 반환

결과 분석

In [15]:
# 2.6. 5단계: 결과 분석 함수
def analyze_results(results):
    """
    튜닝 결과를 요약합니다.
    """
    if not results:
        print("결과가 비어있어 분석을 스킵합니다.")
        return

    # 정확도가 가장 높은 결과를 찾음
    best_result_summary = max(results, key=lambda x: x['accuracy'])
    
    print("\n--- 모든 튜닝 결과 ---")
    # 정확도 순으로 정렬하여 보기 좋게 출력
    for res in sorted(results, key=lambda x: x['accuracy'], reverse=True):
        print(f"K: {res['k']:<3} | "
              f"Weights: {res['weights']:<10} | "
              f"분산: {res['variance']:.2f} | "
              f"차원: {res['n_components']:<4} | "
              f"정확도: {res['accuracy'] * 100:.2f}%")
              
    print("\n--- 🏆 가장 좋은 결과 (요약) ---")
    print(f"최적 K: {best_result_summary['k']}")
    print(f"최적 Weights: {best_result_summary['weights']}")
    print(f"최적 분산 비율: {best_result_summary['variance']}")
    print(f"최적 차원 수: {best_result_summary['n_components']}")
    print(f"최고 정확도: {best_result_summary['accuracy'] * 100:.2f}%")

메인 파이프라인 실행

In [16]:
# --- 1. 데이터 로드 ---
X_train, y_train, X_test, y_test = load_and_split_data()

# --- 2. 데이터 증강 ---
X_train_aug, y_train_aug = augment_data(X_train, y_train)

# --- 3. 스케일링 ---
X_train_scaled, X_test_scaled = scale_data(X_train_aug, X_test)

# --- 4. 튜닝 실행 ---

# ⭐️ 튜닝할 하이퍼파라미터 설정 ⭐️
variance_ratios_to_test = [0.65, 0.75, 0.85, 0.95]
weights_to_test = ['uniform', 'distance']
k_values_to_test = [2, 4, 6, 8]

results = tune_pca_and_knn(
    X_train_scaled, 
    y_train_aug, 
    X_test_scaled, 
    y_test,
    variance_ratios_to_test,
    weights_to_test,
    k_values_to_test
)

# --- 5. 결과 분석 ---
analyze_results(results)

print("\n--- 모든 작업 완료 ---")

MNIST 전체 데이터셋 로드 중...
원본 훈련 세트: 60000개, 테스트 세트: 10000개

데이터 증강 시작...
  ... (dy=0, dx=1) 이동 적용 중 ...
  ... (dy=0, dx=-1) 이동 적용 중 ...
  ... (dy=1, dx=0) 이동 적용 중 ...
  ... (dy=-1, dx=0) 이동 적용 중 ...
총 훈련 세트 크기: 300000개

데이터 스케일링 중...
스케일링 완료.

--- PCA 및 KNN 그리드 탐색 시작 ---
후보 분산: [0.65, 0.75, 0.85, 0.95]
후보 Weights: ['uniform', 'distance']
후보 K: [2, 4, 6, 8]
(총 32회 실행. 매우 오래 걸립니다.)

[ Weights = 'uniform', 분산 = 0.65 ] 테스트 시작...
  PCA 적용 (n_components=0.65)...
  PCA 완료. (차원 수: 84개)
  [ 1/32 ] k=2 모델 훈련/예측 중...
  KNN 완료.
  >>> (k=2) 정확도: 96.15% <<<
  *** 새 최고 정확도 달성! ***
  [ 2/32 ] k=4 모델 훈련/예측 중...
  KNN 완료.
  >>> (k=4) 정확도: 96.91% <<<
  *** 새 최고 정확도 달성! ***
  [ 3/32 ] k=6 모델 훈련/예측 중...
  KNN 완료.
  >>> (k=6) 정확도: 96.88% <<<
  [ 4/32 ] k=8 모델 훈련/예측 중...
  KNN 완료.
  >>> (k=8) 정확도: 96.94% <<<
  *** 새 최고 정확도 달성! ***

[ Weights = 'uniform', 분산 = 0.75 ] 테스트 시작...
  PCA 적용 (n_components=0.75)...
  PCA 완료. (차원 수: 127개)
  [ 5/32 ] k=2 모델 훈련/예측 중...
  KNN 완료.
  >>> (k=2) 정확도: 96.14% <<<
  [ 6/32 ] k=4 모